# Funcitonality
RNN is a class of nets that can predict the future. You can use it on sequences of arbitrary length, rather than fixed-sized inputs. So you can take sentences, documents or audios as input and make natural language precessing system like translation, speech to text or sentiment analysis(reading movie reviews and extracting the rate)

You can also use RNN more creatively. Ask it to predit which are the most likely next notes in a melody and randomly pick one and play it. Then ask for the next note.

# Simple Structure 

At each time step(frame), the recurrent neuron receives the inputs $x_{(t)}$ as well as the output from the previous time step,$y_{(t-1)}$
$$ y_{(t)} =\phi(x_{t}^T·w_x+y_{(t-1)}^T·w_y+b)$$

Since the output of each neutron is a function of all inputs from previous time step, is has a form of memory.

In [198]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data

# Simple RNN from srcatch
<img src='rnn.jpeg' width=300 />

In [61]:
n_inputs = 3
n_neurons = 5

In [62]:
x0 = tf.placeholder(tf.float32,[None,n_inputs])
x1 = tf.placeholder(tf.float32,[None,n_inputs])

In [63]:
wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
wy = tf.Variable(tf.random_normal(shape=[n_neuron,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32)) 

In [64]:
y0 = tf.tanh(tf.matmul(x0,wx)+b)

In [65]:
y1 = tf.tanh(tf.matmul(x1,wx)+tf.matmul(y0,wy)+b)

In [66]:
init = tf.global_variables_initializer()

In [67]:
x0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
x1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])


In [68]:
with tf.Session() as sess:
    init.run()
    y0_val,y1_val = sess.run([y0,y1],feed_dict={x0:x0_batch,x1:x1_batch})

In [69]:
y0_val

array([[ 0.817643  ,  0.99002373, -0.15017775, -0.99039215, -0.11675904],
       [ 0.6172834 ,  0.9609914 ,  0.7072065 , -0.9999998 , -0.99672025],
       [ 0.28355524,  0.8536817 ,  0.9574591 , -1.        , -0.9999932 ],
       [-0.99999976, -1.        ,  0.9999995 , -0.9999973 , -1.        ]],
      dtype=float32)

In [70]:
y1_val

array([[-0.99992687, -0.99934644,  0.9561947 , -1.        , -0.99999994],
       [-0.9871589 ,  0.38692433,  0.3511038 , -0.2169611 , -0.7101225 ],
       [-0.9988916 , -0.9993261 ,  0.99648106, -1.        , -0.99999815],
       [-0.45749965, -0.99992913,  0.99873984, -0.99572533, -0.99713975]],
      dtype=float32)

# RNN in tensorflow

In [56]:
tf.reset_default_graph()
n_steps = 2
n_inputs = 3
n_neurons = 5

In [57]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])

In [58]:
x_seqs = tf.unstack(tf.transpose(x,perm=[1,0,2]))

In [59]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,reuse=tf.AUTO_REUSE)

In [27]:
output_seqs, state = tf.contrib.rnn.static_rnn(basic_cell,x_seqs,dtype=tf.float32) 

In [28]:
outputs = tf.transpose(tf.stack(output_seqs),perm=[1,0,2])

In [32]:
init = tf.global_variables_initializer()

In [33]:
file_write = tf.summary.FileWriter('./rnn',tf.get_default_graph())

In [71]:
x_batch = np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]],    
])

In [35]:
with tf.Session() as sess:
    init.run()
    a = sess.run(x_seqs,feed_dict={x:x_batch})
    output_val = outputs.eval(feed_dict={x:x_batch})

In [36]:
output_val

array([[[ 0.7056197 ,  0.74873227,  0.8752992 , -0.63706374,
          0.4218465 ],
        [ 0.9992527 ,  0.4342795 ,  1.        ,  0.8697232 ,
          0.9767722 ]],

       [[ 0.98771864,  0.9411917 ,  0.99985963, -0.65383077,
          0.880359  ],
        [-0.06295659, -0.7625946 ,  0.20157966,  0.7458381 ,
         -0.17764854]],

       [[ 0.99955773,  0.9873063 ,  0.99999976, -0.6699795 ,
          0.9802841 ],
        [ 0.98396033, -0.3915091 ,  0.99997765,  0.9398496 ,
          0.83627725]],

       [[-0.9044139 , -0.761997  ,  0.9998387 ,  0.83228326,
         -0.9693899 ],
        [ 0.2948277 , -0.49465013,  0.884769  ,  0.7809286 ,
          0.80485773]]], dtype=float32)

In [37]:
file_write.close()

Above, there are one time step. If we have larger number of time steps, the graph won't look good. And we may get out-of-memory problem since it must store all tensor values during the forward and backward pass. 

# There is a better solution: dynamic_rnn() 
There is no need to stack, unstack or transpose the input.


In [80]:
tf.reset_default_graph()

In [81]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])

In [82]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,reuse=tf.AUTO_REUSE)

In [83]:
outputs,states = tf.nn.dynamic_rnn(basic_cell,x,dtype=tf.float32)

In [84]:
init = tf.global_variables_initializer()

In [85]:
with tf.Session() as sess:
    sess.run(init)
    outputs = sess.run(outputs,feed_dict={x:x_batch})

In [86]:
outputs

array([[[ 0.29133332, -0.40971285,  0.00474932,  0.7817908 ,
         -0.17873375],
        [-0.9999993 , -1.        , -0.9999995 ,  0.84270215,
         -0.6409403 ]],

       [[-0.9725871 , -0.99794674, -0.99233806,  0.9329037 ,
         -0.49224204],
        [-0.3418065 , -0.09544241, -0.3896178 , -0.8752379 ,
          0.9042722 ]],

       [[-0.999788  , -0.9999951 , -0.99997073,  0.9805134 ,
         -0.7150079 ],
        [-0.9999651 , -0.9999829 , -0.99998575, -0.84973294,
          0.77066094]],

       [[-0.99995595, -0.9998664 , -0.99997795, -0.99973094,
          0.8243463 ],
        [-0.99960214, -0.58785963, -0.99986583, -0.98049116,
          0.8156727 ]]], dtype=float32)

# Variable length input sequence
If the input sequences of each instance is different, we have to clarity the number of sequence for each instance by setting the parameter  <code>sequence_length</code> when calling <code>python dynamic_rnn()</code>

In [189]:
tf.reset_default_graph()
n_steps = 2
n_inputs = 1
n_neurons = 1
x_batch=[
    [[2],[0]], # the dim of sequence in this should be the same with n_step. If there is no enough squence,pad with zero
    [[1],[1]]
]

In [190]:
x = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
seq_length = tf.placeholder(tf.int32,[None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,reuse=tf.AUTO_REUSE)
outputs, states = tf.nn.dynamic_rnn(basic_cell,x,dtype=tf.float32,sequence_length=seq_length)

init = tf.global_variables_initializer()

In [191]:
seq_length_batch = np.array([1,2]) # each of the number in this array should be less or equal at n_steps

In [192]:
with tf.Session() as sess:
    sess.run(init)
    outputs_val, states_val = sess.run([outputs,states],feed_dict={x:x_batch,seq_length:seq_length_batch})

In [193]:
outputs_val

array([[[-0.9169307 ],
        [ 0.        ]],

       [[-0.6553968 ],
        [-0.53586686]]], dtype=float32)

In [121]:
states_val

array([[0.9570105]], dtype=float32)

# Train RNN to MINST Data 

In [261]:
home_path = os.getenv('HOME')
data_path = home_path+'/program/ml/datasets/MNIST_data'

In [262]:
data = input_data.read_data_sets(data_path)

Extracting /Users/zyan/program/ml/datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/zyan/program/ml/datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/zyan/program/ml/datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/zyan/program/ml/datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


In [300]:
tf.reset_default_graph()
n_inputs = 28
n_steps = 28
n_neurons = 100
n_outputs = 10
learning_rate = 0.001
x = tf.placeholder(tf.float32,shape=[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None])
basic_cell = tf.nn.rnn_cell.BasicRNNCell(n_neurons,reuse=tf.AUTO_REUSE)
#basic_cell = tf.nn.rnn_cell.DropoutWrapper(basic_cell,input_keep_prob=0.5)# add dropout

outputs, stats = tf.nn.dynamic_rnn(basic_cell,inputs=x,dtype=tf.float32,initial_state=)
logits = tf.layers.dense(stats,n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits) 
loss = tf.reduce_mean(xentropy)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1) 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 
init = tf.global_variables_initializer()

In [301]:
n_epochs = 100
batch_size = 1000

In [302]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        n_batch = data.train.num_examples//batch_size
        for batch in range(n_batch):
            x_batch,y_batch = data.train.next_batch(batch_size)
            x_batch = x_batch.reshape((-1,n_steps,n_inputs))
            sess.run(train_op,feed_dict={x:x_batch,y:y_batch})
        accuracy_train = accuracy.eval(feed_dict={x:data.train.images.reshape((-1,n_steps,n_inputs)),y:data.train.labels})
        accuracy_test = accuracy.eval(feed_dict={x:data.test.images.reshape((-1,n_steps,n_inputs)),y:data.test.labels})
        print(epoch,'train_accuracy:',accuracy_train,'test_accuracy:',accuracy_test)
    #outputs_eval = outputs.eval(feed_dict={x:x_batch[:10]})
    #stats_eval = stats.eval(feed_dict={x:x_batch[:10]})
    

0 train_accuracy: 0.6696 test_accuracy: 0.6786
1 train_accuracy: 0.7611455 test_accuracy: 0.7721
2 train_accuracy: 0.8085273 test_accuracy: 0.8119
3 train_accuracy: 0.83885455 test_accuracy: 0.8459
4 train_accuracy: 0.85703635 test_accuracy: 0.8622
5 train_accuracy: 0.86663634 test_accuracy: 0.8715
6 train_accuracy: 0.8832909 test_accuracy: 0.8862
7 train_accuracy: 0.8898909 test_accuracy: 0.8939
8 train_accuracy: 0.8949818 test_accuracy: 0.8964
9 train_accuracy: 0.90254545 test_accuracy: 0.9038
10 train_accuracy: 0.9065091 test_accuracy: 0.9091
11 train_accuracy: 0.9055091 test_accuracy: 0.9114
12 train_accuracy: 0.9166727 test_accuracy: 0.9219
13 train_accuracy: 0.9164 test_accuracy: 0.9198
14 train_accuracy: 0.92163634 test_accuracy: 0.9218
15 train_accuracy: 0.9197818 test_accuracy: 0.9227
16 train_accuracy: 0.9263091 test_accuracy: 0.9295
17 train_accuracy: 0.9206727 test_accuracy: 0.9198
18 train_accuracy: 0.93087274 test_accuracy: 0.9292
19 train_accuracy: 0.9304364 test_accurac